# Lista de exercícios 07

## Imports default

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import linear_model as lm
from sklearn import metrics
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import svm

### Datasets

In [2]:
from sklearn.datasets import load_boston, load_breast_cancer

# Regressão

## separando dados e label

In [3]:
X, y = load_boston(return_X_y=True)
X.shape

(506, 13)

## Aplicando Standadization no dados

In [4]:
X_scaled = preprocessing.StandardScaler().fit(X)
X_scaled

StandardScaler(copy=True, with_mean=True, with_std=True)

## Aplicando k-fold

In [5]:
kf = KFold(n_splits=5)

## Gradient Descent

In [14]:
erros_mse_gradient = np.empty((1,0))
erros_mae_gradient = np.empty((1,0))
gd = lm.SGDRegressor()

for train_index,test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gd.fit(X_train.astype(float),y_train.astype(float))
    ŷ = gd.predict(X_test)
    erros_mse_gradient = np.append(erros_mse_gradient, np.sqrt(metrics.mean_squared_error(y_test, ŷ)))
    erros_mae_gradient = np.append(erros_mae_gradient, metrics.mean_absolute_error(y_test, ŷ))
    
erros_mse_gradient, erros_mae_gradient

/home/eudalio/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


(array([1.41679888e+14, 1.68453060e+14, 3.46491383e+14, 3.44338126e+14,
        2.02086809e+14]),
 array([1.33833418e+14, 1.62710102e+14, 3.43633374e+14, 3.40229380e+14,
        1.95649812e+14]))

## Regressão Linear

In [13]:
erros_mse = np.empty((1,0))
erros_mae = np.empty((1,0))
modelo = lm.LinearRegression()

for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]  
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    erros_mse = np.append(erros_mse, np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    erros_mae = np.append(erros_mae, metrics.mean_absolute_error(y_test, y_pred))

erros_mse, erros_mae

(array([3.53279637, 5.10844425, 5.75499399, 8.99072043, 5.79520109]),
 array([2.62469524, 3.91244817, 4.38945068, 5.57366377, 4.78261237]))

## KNN

In [12]:
erros_mse_knn = np.empty((1,0))
erros_mae_knn = np.empty((1,0))
knn = neighbors.KNeighborsRegressor()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    knn.fit(X_test, y_test)
    ŷ = knn.predict(X_test)
    erros_mse_knn = np.append(erros_mse_knn, np.sqrt(metrics.mean_squared_error(y_test, ŷ)))
    erros_mae_knn = np.append(erros_mae_knn, metrics.mean_absolute_error(y_test, ŷ))
erros_mse_knn, erros_mae_knn

(array([3.7574329 , 4.25541891, 5.90849967, 5.98375457, 3.12392049]),
 array([2.6927451 , 2.87188119, 4.47128713, 3.80574257, 2.35425743]))

## Naive bayes

In [11]:
erros_mse_nb = np.empty((1,0))
erros_mae_nb = np.empty((1,0))
nb = naive_bayes.GaussianNB()
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    nb.fit(X_test.astype(float), y_test.astype(int))
    ŷ = nb.predict(X_test.astype(float))
    erros_mse_nb = np.append(erros_mse_nb, np.sqrt(metrics.mean_squared_error(y_test, ŷ)))
    erros_mae_nb = np.append(erros_mae_nb, metrics.mean_absolute_error(y_test, ŷ))
    
erros_mse_nb, erros_mae_nb

(array([2.32629892, 3.35857112, 3.15381272, 2.7787026 , 3.94671188]),
 array([1.42058824, 1.6990099 , 1.69306931, 1.65148515, 2.54158416]))

## SVM

In [10]:
erros_mse_svm = np.empty((1,0))
erros_mae_svm = np.empty((1,0))
svm = svm.SVR()
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svm.fit(X_train.astype(float), y_train.astype(int))
    ŷ = svm.predict(X_test.astype(float))
    erros_mse_svm = np.append(erros_mse_svm, np.sqrt(metrics.mean_squared_error(y_test, ŷ)))
    erros_mae_svm = np.append(erros_mae_svm, metrics.mean_absolute_error(y_test, ŷ))
erros_mse_svm, erros_mae_svm

(array([ 6.12783971, 10.19544902, 13.60394648,  9.38622491,  7.74517624]),
 array([ 4.38970324,  6.98192915, 10.30474094,  6.59267467,  6.4220285 ]))